# Proyecto de Criptografía y Seguridad: Análisis del paper "Computing Arbitrary Functions of Encrypted Data" de Craig Gentry  


Introducción: 

Un problema clave en la era de la computación en la nube es la privacidad, pues en una época donde dependemos enormemente del mundo digital, escándalos como el de Cambridge Analytica nos hace darnos cuenta que el manejo que grandes conglomerados como Facebook le pueden hacer a nuestros datos puede tener detrás la intención de manipularnos... 

Craig Gendry nos hace la pregunta, ¿es posible delegar el procesamiento de datos sin exponer su contenido? 

En el presente trabajo analizaremos y entenderemos la propuesta que da Gendry como respuesta a la pregunta y presentaremos unas simples implementaciones de dicha propuesta...

Empezaremos dando la definición de operación homomórfica...

Las operaciones homomórficas son aquellas que operaciones que pueden realizarse sobre datos encriptados de forma que al obtener el resultado de hacer dichas operaciones y desencriptarlo, coincida al aplicar las mismas operaciones sobre el texto plano. Matemáticamente significa que si $E(m)$ es la encripción del mensaje $m$, una operación $∘$ es homomórfica si $E(m_{1})∘E(m_{2}) = E(m_{1}∘m_{2})$, es decir, operar sobre datos cifrados equivale a operar sobre los datos originales antes de cifrarlos. Algunos ejemplos de operaciones homomórficas son las operaciones algebraicas suma y multiplicación; operaciones lógicas como AND, OR, XOR, NOT y en algunos casos operaciones de comparación. Alugnos ejemplos de funciones no homomórficas son la división, la raíz cuadrada, logaritmos y funciones trigonométricas... 

Continuamos definiendo el Cifrado Homomórfico: 

Según los autores Nimish Jain y Aswani Kumar Cherukuri ellos defininen en su paper "Revisiting Fully Homomorphic Encryption Schemes" el Cifrado Homomórfico como: El cifrado homomórfico es una técnica de cifrado sofisticada que permite realizar cálculos sobre datos cifrados sin necesidad de descifrarlos. Esta característica hace que el cifrado homomórfico sea adecuado para realizar cálculos seguros en escenarios de datos sensibles, como la computación en la nube, el intercambio de datos médicos y las transacciones financieras. Los datos se cifran utilizando una clave pública en el cifrado homomórfico y el cálculo se realiza sobre los datos cifrados utilizando un algoritmo que conserva el cifrado. El resultado calculado se descifra luego con una clave privada para obtener el resultado final. Esta noción abstracta protege los datos al tiempo que permite realizar cálculos complicados sobre los datos cifrados, lo que da como resultado un enfoque seguro y eficiente para analizar información sensible. Este artículo tiene como objetivo dar una idea clara sobre los diversos esquemas de cifrado totalmente homomórfico presentes en la literatura y analizar y comparar los resultados de cada uno de estos esquemas. Además, también proporcionamos aplicaciones y herramientas de código abierto de esquemas de cifrado homomórfico. 

Una definicón más rigurosa de cifrado Homomórfico sería la siguiente:

Un esquema de cifrado homomórfico (HE) es un tupla de algoritmos: $HE = (KeyGen, Encrypt, Decrypt, Evaluate)$ donde, 

1. $KeyGen(\lambda) \rightarrow (pk, sk) \\$ 
-Genera un par de claves: clave pública $pk$ y clave privada $sk \\$ 
-$\lambda$ es el parámetro de seguridad. $\\$
2. $Encrypt(pk, m) \rightarrow c \\$ 
-Toma la llave pública $pk$ y un mensaje $m$ del espacio de Mensajes $M \\$ 
-Devuelve un cifrado $c$ en el espacio de mensajes cifrados $C \\$  
3. $Decrypt (sk, c) \rightarrow m \\$ 
-Toma la clase secreta $sk$ y un cifrado $c \\$ 
-Devuelve el mensaje original $m \\$
4. $Evaluate (pk, f, c_{1}, c_{2}, ..., c_{t}) \rightarrow c_{f} \\$ 
-Toma la clave pública, una función computable y una colección de cifrados $c_{1}, c_{2}, ..., c{t} \\$ 
-Devuelve el nuevo cifrado $c_{f}$ tal que: $Decrypt(sk, c_{f}) = f(m_{1}, ..., m_{t})$, i.e. el resultado cifrado de aplicar $f$ sobre los textos planos. 

Existen tres tipos de Cifrado Homomórfico: 

1. Cifrado Parcialmente Homomórfico (PHE): El PHE permite realizar solo una operación homomórfica, pudiendo ser la suma o la multiplicación sobre los datos encriptados pero no ambas. 
2. Cifrado Algo-Homomórfico (Somewhat Homomorphic Encryption, SHE): El SHE permite realizar un número limitado de operaciones homomórficas sobre los datos encriptados. 
3. Cifrado Totalmente Homomórfico (Fully Homomorphic Encryption, FHE): El FHE permite realizar un número arbitrario de operaciones homomórficas sobre los datos cifrados. 

Gendry introduce un esquema de Cifrado Totalmente Homomórfico (FHE), que puede evaluar cualquier función sin restricciones, dicho esquema se construye en dos pasos fundamentales:

1. Construcción de un Cifrado Algo-Homomórfico (SHE): Gendry comienza diseñando un cifrado SHE que permita un número limitado de sumas y multiplicaciones en el mensaje encriptadas; se empieza con un número limitado de operaciones ya que cada operación homomórfica sobre un mensaje encriptado genera un ruido y si el ruido se acumula demasiado el mensaje cifrado se vuelve ilegible y no podría ser descrifrado correctamente, i.e, $c=Encrypt(m)+$ ruido, metiante se vayan haciendo operaciones el ruido crece hasta que el mensaje se vuelve irrecuperable.   
2. Bootstrapping: Para solucionar el problema del ruido, Gendry introduce un truco brillante que lo nombra como Bootstrapping; consiste en usar el propio cifrado homomórfico para descrifrar parcialmente el mensaje, eliminar el ruido existente sin revelar el contenido y luego de eliminar el ruido se puede operar con el mensaje cifrado sin el ruido. Descifrar parcialmente quiere decir que en lugar de descifrar el mensaje de tal forma que quede un texto plano, se evalúa el algoritmo de descrifrado como una función homomórfica, es decir que para un texto plano $m$, $E(m) \rightarrow Evaluate(Decrypt, E(m))$, esto genera un nuevo cifrado $E'(m)$ con menos ruido y con $E'(m)$ se sigue operando. El ruido se vuelve problematico cuando alcanza un valor cercano al tamaño del módulo cifrado, i.e. Si el módulo de cifrado es $N$, entonces el ruido no puede superar el valor de $N$ para que el mensaje se pueda recuper correctamente.   

En el siguiente ejemplo se hará paso por paso un ejemplo sencillo que usa el esquema de Gendry: 

1. Definir el esquema: Dado un número pequeño $m$, cifraremos dicho número utilizando una versión simplificada del esquema de Gendry, para ello tenemos que:
    
   -Módulo $N$: Donde ocurrirán todas las operaciones, en nuestro caso particular $N=23 \\$
   -Clave secreta $q$: Número secreto que se usará en el esquema, en nuestro caso particular $q=5 \\$
   -Ruido inicial $e$: Un pequeño valor aleatorio para ocultar el texto plano, en nuestro caso particular será $e=2\\$ 
   El cifrado se construye como: $c = (m + pq + e) mod N$ donde $p$ es un número grande y aleatorio que no conocemos y que el sistema lo utilizará internamente. $\\$ 

   Si $m_{1}=7$ y $p=3$ tendríamos que $c_{1}=[7+(5 \cdot 3)+2] mod 23 ≡ 1 mod 23$ lo que significa que el cifrado de $7$ es $c_{1}=1\\$  
   Si $m_{2}=4$ con el mismo $p$ tendríamos que $c_{2}=[4+(5 \cdot 3)+2] mod 23 ≡ 17 mod 23$ lo que significa que el cifrafo de $4$ es $c_{2}=17 \\$ 

2. Operaciones: Observemos que $c_{3}=(c{1} + c_{2}) mod N ≡ 18 mod 23$ y que $e_{3} = 5$, todavía se puede seguir operando o desencriptar sin problemas ya que el ruido aún sigue siendo pequeño. 

   Note que $c_{4}=(c_{1} \cdot c_{2}) mod N ≡ 17 mod 23$ y que $e_{4} = 6$, al multiplicar el ruido se acercó al tamaño del módulo $N$, lo que hace que después de varias multiplicaciones y mediante se vaya acercardo al tamaño del módulo $N$ el descrifrado empiece a fallar. 

   Sea $c_{5} = (c_{4} \cdot c_{2}) mod N = 289 mod 23 ≡ 14 mod 23$, $e_{5}=18$, ahora el ruido es $18$ cada vez más cerca de $23$ por lo que si seguimos realizando operaciones perderemos la capacidad de decifrar.   

3. Solución del ruido con Bootstraping: Para poder realizar el desencriptado parcial dentro del mismo sistema, necesitamos almacenar la clave secreta $q$ de manera cifrada, es decir, $E(q) = [q + (p \cdot r) \div e_{q}] mod N$ donde $p = 3$, $r = 4$ aleatorio, $e_{q}=1$ pequeño ruido agregado, quedando así que $E(q) = [q + (p \cdot r) \div e_{q}] mod N ≡ 18 mod 23$. Ahora tenemos $E(q)=18$ lo que significa que ahora la clave secreta está cifrada dentro del mismo sistema. 

   Continuamos descrifrando parcialmente el texto cifrado de la siguiente manera: $E(m) = [(c_{5} - e_{5}) ÷ E(q)] Mod N$ que para nuestro caso particular tenemos que $E(m) = [(14- 18) ÷18] Mod 23 ≡ 19 mod 23$, obteniendo así un nuevo cifrado de $m$ con mucho menos ruido. 

   El resultado de la operación anterior es un mensaje decifrado parcialmente, ahora se vuelve a cifrar desde cero con un ruido nuevo y pequeño. Ahora se vuelve a cifrar desde $0$ con un ruido muy pequeño, i.e. $c' = 19 + (p' \cdot r') + e'$ y se vuelven a elegir valores para $p', r', e'$ que en nuestro caso serán los siguientes,  $p' = 2$, $r' = 3$, $e' = 2$, obteniendo así que $c' = [19 + (2 \cdot 3) + 2] Mond 23 ≡ 2 mod 23 $. Obteniendo así un nuevo cifrado con un ruido pequeño y operando con este nuevo cifrado. 


Algunas desventajas del esquema de Gendry son: 

1. El alto costo computacional, pues las operaciones homomórficas y Bootstrapping son muy lentos lo que hace que FHE no sea viable en la mayoría de las aplicaciones. 
2. Tamaño del cifrado: Un mensaje cifrado puede ser extremadamente más grande que el texto plano original, lo que hace que almacenar y transmitir datos cifrados sea muy costoso.
3. Complejidad matemática: Para usar FHE se requieren avanzados conocimientos de redes Euclidianas y álgebra abstracta, lo que lo hace difícil de implementar y optimizar. 
4. Poca aplicabilidad prática: No es un sistema de encripción útil para aplicaciones donde apremia la velocidad, por lo que hasta el momento el FHE es dificil de integrar en sistemas del mundo real. 

La seguridad que ofrece el sistema de Gendry es de una alta calidad, pues posee indistnguibilidad, el esquema se basa en redes Euclidianas o Lattices lo cuál lo hace extremadamente dificil de decifrar y esto a su vez proporciona una gran resistencia a ataques cuánticos.  














